In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from typing import List
from depsurf import FuncEntry

from depsurf import output, VERSION_LAST

import pandas as pd


class Functions:
    def __init__(self, data=None):
        self.data: List[FuncEntry] = data if data is not None else []

    @classmethod
    def from_jsonl(cls, path):
        with open(path) as f:
            data = [FuncEntry.from_json(line) for line in f]
        return cls(data)

    def get_df(self):
        return pd.DataFrame([f.to_dict() for f in self.data])

In [3]:
funcs = Functions.from_jsonl(VERSION_LAST.funcs_path)

In [4]:


for func in funcs.data:
    if func.has_inline_caller:
        if not func.inline_actual:
            func.print()

    # if (
    #     func.has_func_caller  
    #     and func.has_inline_caller
    #     and func.external is True
    # ):
    #     # func.print()
    #     cnt += 1

# print(cnt)

strlcat
	External: True
	Inline: not declared and not inlined
	Loc: include/linux/fortify-string.h:319
	File: init/main.c
	Caller Inline
		d:o:_:o:n:e:_:i:n:i:t:c:a:l:l:@:i:n:i:t:/:m:a:i:n:.:c
		s:h:o:w:_:t:r:i:g:g:e:r:@:a:r:c:h:/:x:8:6:/:k:e:r:n:e:l:/:c:p:u:/:m:c:e:/:d:e:v:-:m:c:e:l:o:g:.:c
		e:v:e:n:t:_:e:n:a:b:l:e:_:r:e:a:d:@:k:e:r:n:e:l:/:t:r:a:c:e:/:t:r:a:c:e:_:e:v:e:n:t:s:.:c
		e:v:e:n:t:_:e:n:a:b:l:e:_:r:e:a:d:@:k:e:r:n:e:l:/:t:r:a:c:e:/:t:r:a:c:e:_:e:v:e:n:t:s:.:c
		c:r:e:a:t:e:_:f:i:e:l:d:_:v:a:r:_:h:i:s:t:@:k:e:r:n:e:l:/:t:r:a:c:e:/:t:r:a:c:e:_:e:v:e:n:t:s:_:h:i:s:t:.:c
		c:r:e:a:t:e:_:f:i:e:l:d:_:v:a:r:_:h:i:s:t:@:k:e:r:n:e:l:/:t:r:a:c:e:/:t:r:a:c:e:_:e:v:e:n:t:s:_:h:i:s:t:.:c
		c:r:e:a:t:e:_:f:i:e:l:d:_:v:a:r:_:h:i:s:t:@:k:e:r:n:e:l:/:t:r:a:c:e:/:t:r:a:c:e:_:e:v:e:n:t:s:_:h:i:s:t:.:c
		c:r:e:a:t:e:_:f:i:e:l:d:_:v:a:r:_:h:i:s:t:@:k:e:r:n:e:l:/:t:r:a:c:e:/:t:r:a:c:e:_:e:v:e:n:t:s:_:h:i:s:t:.:c
		c:r:e:a:t:e:_:f:i:e:l:d:_:v:a:r:_:h:i:s:t:@:k:e:r:n:e:l:/:t:r:a:c:e:/:t:r:a:c:e:_

NameError: name 'cnt' is not defined

In [ ]:
# df = funcs.get_df()

# df[df["name"].str.contains("vfs_")]